## FINAL CODE OF WEBSCRAPING PROJECT

### Importing all routines

In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import time
import os
from bs4 import BeautifulSoup as bs
import pandas as pd
import unicodedata 
import numpy as np
import requests
from selenium.common.exceptions import NoSuchElementException
import re
import tkinter as tk
from tkinter import *
import customtkinter
import csv
import webbrowser

### Saving some global functions, used to clean DataFrame

In [14]:

# get data from a container
def get_element(key, html):
    element = html.select(key)
    temp = []
    if len(element) > 0:
        return element[0].text.replace('\n', '').replace('  ', '')
    else:
        return np.nan

# clean \x0
def clean_x(s):                                                
    clean_text = unicodedata.normalize("NFKD",s)
    clean_text = clean_text.replace(" ", "")
    return clean_text    

# clean \x0
def clean_x1(s):                                                
    clean_text = unicodedata.normalize("NFKD",s)
    #clean_text = clean_text.replace(" ", "")
    return clean_text 


#   This function is used later to convert the cooktime-values from EX: 1H 30 MIN to 90
def convert_to_minutes(time_str):
    if pd.isna(time_str) or time_str == "":
        return np.nan
    
    match = re.findall(r'\d+', time_str)
    if len(match) == 2:
        hours, minutes = match
        total_minutes = int(hours) * 60 + int(minutes)
    elif len(match) == 1:
        if 'H' in time_str:
            total_minutes = int(match[0]) * 60
        elif 'M' in time_str:
            total_minutes = int(match[0])
        else:
            return np.nan
    else:
        return np.nan
    
    return total_minutes

def convert_to_integer(value_str):
    if isinstance(value_str, str):
        match = re.findall(r'\d+', value_str)
        if match:
            return int(match[0])
    return None 

### WEBSCRAPING RECIPE CATALOG, SAVING DF

In [15]:
# create a web driver and go to page
driver = webdriver.Chrome() 
driver.get("https://lifesum.com/nutrition-explained/recipes/") 

# cookie related buttom

WebDriverWait(driver, 2)                                               
# wait 2 sec 

cookie_option = driver.find_element(By.XPATH, '//*[@id="mount-modal"]/div/div/div/div[2]/div/div[2]/button[1]') 
# find the location of the bottom

cookie_option.click()

recipe_url_lst = [] # Create a list to store the unique links
recipe_names_lst = [] # Create a list to store the unique recipe names
recipe_cooktimes_lst = [] # Corresponding cook time for each recipe


for i in range(0, 100000):

    
    try: 
        show_more_recipes = driver.find_element(By.XPATH, '//*[@id="recipes"]/div/div[2]/button')
        show_more_recipes.click()
        time.sleep(1)
    except (NameError, NoSuchElementException) as error:
        break
        

for j in range(0,228):
# Collect the links after clicking the button
    recipe_links = driver.find_elements(By.XPATH, f'//*[@id="recipes"]/div/div[1]/div[{j}]/div[2]/a')
    for link in recipe_links:
        if link not in recipe_url_lst:
            recipe_url_lst.append(link.get_attribute("href")) # Use add() method instead of append()

    recipe_names = driver.find_elements(By.XPATH, f'//*[@id="recipes"]/div/div[1]/div[{j}]/div[2]/a/h2/span')
    for name in recipe_names:
        recipe_name = name.text
        if recipe_name not in recipe_names_lst and name in recipe_names:
            recipe_names_lst.append(recipe_name)
        else:
            recipe_names_lst.append(np.nan)
          
    recipe_cooktimes = driver.find_elements(By.XPATH, f'//*[@id="recipes"]/div/div[1]/div[{j}]/div[2]/div/ul')
    for cooktime in recipe_cooktimes:
        try:
            recipe_cooktime = cooktime.text
            recipe_cooktimes_lst.append(f"{recipe_cooktime}")
        except (NoSuchElementException) as error:
            recipe_cooktimes_lst.append(np.nan)
            
# Creating the dataframe, naming columns, cleaning column values

df = pd.DataFrame([recipe_names_lst, recipe_url_lst, recipe_cooktimes_lst]).T

df[['3', '4']] = df[2].str.split('\n', expand=True)

df.rename(columns = {0: "recipe_name", 1:"recipe_url", '3':"cooktime", '4':"kcals"}, inplace=True)

df.drop(columns=(2), inplace=True)
df

df['cooktime(min)'] = df['cooktime'].apply(convert_to_minutes)

df.drop(columns=("cooktime"), inplace=True)

df['kcals'] = df['kcals'].apply(convert_to_integer)

df

,recipe_name,recipe_url,kcals,cooktime(min)
0,Grilled Salmon,https://lifesum.com/nutrition-explained/recipe...,236.0,40.0
1,Sweet Potato Stew,https://lifesum.com/nutrition-explained/recipe...,660.0,30.0
2,Spicy Chicken Bowl,https://lifesum.com/nutrition-explained/recipe...,541.0,20.0
3,Banana & Walnut Shake,https://lifesum.com/nutrition-explained/recipe...,413.0,5.0
4,Apple with Almond Coconut Dip,https://lifesum.com/nutrition-explained/recipe...,182.0,5.0
...,...,...,...,...
222,Carrot Cake Yogurt Parfait,https://lifesum.com/nutrition-explained/recipe...,249.0,70.0
223,Zucchini Pesto Pasta with Cherry Tomatoes and ...,https://lifesum.com/nutrition-explained/recipe...,470.0,45.0
224,Frosted Chocolate and Coconut Shots,https://lifesum.com/nutrition-explained/recipe...,125.0,20.0
225,Raw Mango and Coconut Sorbet,https://lifesum.com/nutrition-explained/recipe...,NaN,10.0


## SAVING RECIPE-SPECIFIC DATA.

## `DO NOT RUN`

In [ ]:
for länk in df["recipe_url"]:
    
    # create a web driver and go to page
    driver = webdriver.Chrome() 
    driver.get(länk) 

    # cookie related buttom
    WebDriverWait(driver, 2)                                               
    # wait 2 sec 

    cookie_option = driver.find_element(By.XPATH, '//*[@id="mount-modal"]/div/div/div/div[2]/div/div[2]/button[1]') 
    # find the location of the bottom

    cookie_option.click()

    ingredients_lst = [] # Create a list for all ingredients in the recipe
    recipe_instructions_lst = [] # Create a list to store the instructions in
    recipe_name_lst = []   # The corresponding name of the recipe, for formatting purposes
    recipe_category_lst = []   # The categorical labels of each recipe

    
    # This section holds the pathway information to all necessary data being scraped.
    check_ingredients = driver.find_elements(By.XPATH, '//*[@id="mount"]/div[2]/div[2]/div[2]/ul')
    recipe_instructions = driver.find_elements(By.XPATH, f'//*[@id="mount"]/div[2]/div[2]/div[3]/ol')
    recipe_name = driver.find_elements(By.XPATH, '//*[@id="mount"]/div[2]/div[1]/div/div/div/h1')
    recipe_categories = driver.find_elements(By.XPATH, '//*[@id="mount"]/div[2]/div[2]/div[3]/div/ul')


    # This loop iterates over a div on the website containing all the ingredients. 
    # The ingredients are added to a list and when the div is exhausted, the loop closes.
    for k in check_ingredients:

        try:
            ingredient = k.text
            ingredients_lst.append(ingredient)
        except (NoSuchElementException) as error:
            ingredients_lst.append(np.nan)
            break

            
    # Same thing for instructions
    for l in recipe_instructions:

        try:
            instruction = l.text
            recipe_instructions_lst.append(instruction)
        except (NoSuchElementException) as error:
            recipe_instructions_lst.append(np.nan)
            break

    for m in recipe_name:
        try:
            name = m.text
            recipe_name_lst.append(name)
        except (NoSuchElementException) as error:
            recipe_name_lst.append(np.nan)
            break

    for n in recipe_categories:
        try:
            category = n.text
            recipe_category_lst.append(category)
        except (NoSuchElementException) as error:
            recipe_category_lst.append(np.nan)
            break
            
            
    # Now, all the information on a page has been gathered into corresponding lists.
    # The below section assigns the contents of each list to a string, or a dataframe each.
    
    
    name = recipe_name_lst[0]
    df_recipe = pd.DataFrame(ingredients_lst)[0].str.split('\n', expand=True).T
    df_instructions = pd.DataFrame(recipe_instructions_lst)[0].str.split('\n', expand=True).T
    if len(recipe_category_lst) > 0 and recipe_category_lst[0]:
        df_categories = pd.DataFrame(recipe_category_lst)[0].str.split('\n', expand=True).T
    else:
        df_categories = pd.DataFrame()
    
    

    # Cleans the data a bit, by transposing data and removing unneccesary integer values. 
    df_instructions = df_instructions.iloc[1::2].reset_index(drop=True)

    # Places each dataframe in a csv-file, named after the recipe string.
    df_recipe.to_csv(f"{name}_recipe.csv", index=False)
    df_instructions.to_csv(f"{name}_instructions.csv", index=False)
    df_categories.to_csv(f"{name}_categories.csv", index=False)

    driver.quit()

## GRAPHIC USER INTERFACE

#### CHOOSING A RECIPE!

# def on_option_selected():
    selected_value = selected_option.get()
    if selected_value == 'Ingredients':
        print("Selected option: Ingredients")
        prompt_ingredient_input()
    elif selected_value == 'Cooktime':
        print("Selected option: Cooktime")
        prompt_cooktime_input()
    elif selected_value == 'Calories':
        print("Selected option: Calories")
        prompt_calories_input()


def prompt_ingredient_input():
    ingredient_window = tk.Toplevel(window)
    ingredient_window.title("Ingredient Input")
    prompt_label = tk.Label(ingredient_window, text="Enter an ingredient:")
    prompt_label.pack()
    ingredient_entry = tk.Entry(ingredient_window, width=30)
    ingredient_entry.pack()
    search_button = tk.Button(ingredient_window, text="Search", command=lambda: search_ingredient_in_files(ingredient_entry.get(), ingredient_window))
    search_button.pack()

def search_ingredient_in_files(ingredient, ingredient_window):
    folder_path = '/Users/axelreman/Documents/Nod Coding Bootcamp/Codealongs/Week 5/WEBSCRAPING_API_PROJECT/recipes'

    results = []
    for filename in os.listdir(folder_path):
        if filename.endswith('recipe.csv'):
            dish_name = filename.split('_')[0]
            csv_file_path = os.path.join(folder_path, filename)
            with open(csv_file_path, 'r') as file:
                reader = csv.reader(file)
                for row in reader:
                    recipe_ingredients = row[0]
                    if ingredient.lower() in recipe_ingredients.lower():
                        results.append(dish_name)

    ingredient_window.destroy()
                        
    if results:
        select_dish_window = tk.Toplevel(window)
        select_dish_window.title("Select Dish")

        prompt_label = tk.Label(select_dish_window, text="Select a dish:")
        prompt_label.pack()

        dish_listbox = tk.Listbox(select_dish_window, width=40, height=20)
        dish_listbox.pack()

        for result in results:
            dish_listbox.insert(tk.END, result)

        def on_dish_selected():
            selected_dish = dish_listbox.get(dish_listbox.curselection())
            show_recipe_contents(selected_dish)
            select_dish_window.destroy()

        def show_recipe_contents(recipe_name):
            recipe_csv_file = f"{recipe_name}_recipe.csv"
            recipe_csv_path = os.path.join(folder_path, recipe_csv_file)

            with open(recipe_csv_path, 'r') as file:
                reader = csv.reader(file)
                recipe_contents = list(reader)

            recipe_contents_text = tk.Text(window, width=50, height=10)
            recipe_contents_text.insert(tk.END, f"Dish Name: {recipe_name}\n")
            recipe_contents_text.insert(tk.END, "INGREDIENTS\n")
            
            first_row = recipe_contents[0]
            first_row_modified = [cell.replace('0', ' ') for cell in first_row]
            recipe_contents[0] = first_row_modified
            
            for row in recipe_contents:
                recipe_contents_text.insert(tk.END, ", ".join(row) + "\n")

            recipe_instructions_file = f"{recipe_name}_instructions.csv"
            recipe_instructions_path = os.path.join(folder_path, recipe_instructions_file)

            with open(recipe_instructions_path, 'r') as file:
                reader = csv.reader(file)
                recipe_instructions = list(reader)

            first_row = recipe_instructions[0]
            first_row_modified = [cell.replace('0', ' ') for cell in first_row]
            recipe_instructions[0] = first_row_modified
            
            recipe_contents_text.insert(tk.END, "\nINSTRUCTIONS\n")
            for row in recipe_instructions:
                recipe_contents_text.insert(tk.END, ", ".join(row) + "\n"+" "+"\n")

            recipe_contents_text.pack()

        select_button = tk.Button(select_dish_window, text="Select", command=on_dish_selected)
        select_button.pack()

    else:
        result_label = tk.Label(window, text="No results found.").pack()

    ingredient_window.destroy()

def prompt_cooktime_input():
    cooktime_window = tk.Toplevel(window)
    cooktime_window.title("Cooktime Input")
    min_label = tk.Label(cooktime_window, text="Minimum Cooktime (minutes):")
    min_label.pack()
    min_entry = tk.Entry(cooktime_window, width=30)
    min_entry.pack()
    max_label = tk.Label(cooktime_window, text="Maximum Cooktime (minutes):")
    max_label.pack()
    max_entry = tk.Entry(cooktime_window, width=30)
    max_entry.pack()
    search_button = tk.Button(cooktime_window, text="Search", command=lambda: search_cooktime_in_df(int(min_entry.get()), int(max_entry.get()), cooktime_window))
    search_button.pack()

def search_cooktime_in_df(min_cooktime, max_cooktime, cooktime_window):
    results = df[(df['cooktime(min)'] >= min_cooktime) & (df['cooktime(min)'] <= max_cooktime)]
    display_results(results)
    cooktime_window.destroy()

def prompt_calories_input():
    calories_window = tk.Toplevel(window)
    calories_window.title("Calories Input")
    prompt_label = tk.Label(calories_window, text="Enter a range for calories:")
    prompt_label.pack()
    from_label = tk.Label(calories_window, text="From:")
    from_label.pack()
    from_entry = tk.Entry(calories_window, width=10)
    from_entry.pack()
    to_label = tk.Label(calories_window, text="To:")
    to_label.pack()
    to_entry = tk.Entry(calories_window, width=10)
    to_entry.pack()
    search_button = tk.Button(calories_window, text="Search", command=lambda: search_calories_range(from_entry.get(), to_entry.get(), calories_window))
    search_button.pack()

def search_calories_range(from_value, to_value, calories_window):
    from_value = int(from_value)
    to_value = int(to_value)
    results = df[(df['kcals'] >= from_value) & (df['kcals'] <= to_value)]
    display_results(results)
    calories_window.destroy()

def display_results(results):
    if not results.empty:
        select_dish_window = tk.Toplevel(window)
        select_dish_window.title("Select Dish")
        prompt_label = tk.Label(select_dish_window, text="Select a dish:")
        prompt_label.pack()
        dish_listbox = tk.Listbox(select_dish_window, width=40, height=20)
        dish_listbox.pack()

        for index, row in results.iterrows():
            dish_listbox.insert(tk.END, row['recipe_name'])

        def on_dish_selected():
            selected_dish = dish_listbox.get(dish_listbox.curselection())
            show_recipe_details(selected_dish)
            select_dish_window.destroy()


        select_button = tk.Button(select_dish_window, text="Select", command=on_dish_selected)
        select_button.pack()

    else:
        result_label = tk.Label(window, text="No results found.")
        result_label.pack()

def show_recipe_details(recipe_name):
    folder_path = '/Users/axelreman/Documents/Nod Coding Bootcamp/Codealongs/Week 5/WEBSCRAPING_API_PROJECT/recipes'
    recipe_csv_file = f"{recipe_name}_recipe.csv"
    recipe_csv_path = os.path.join(folder_path, recipe_csv_file)

    with open(recipe_csv_path, 'r') as file:
        reader = csv.reader(file)
        recipe_contents = list(reader)

    recipe_contents_text = tk.Text(window, width=50, height=10)
    recipe_contents_text.insert(tk.END, f"Dish Name: {recipe_name}\n")
    recipe_contents_text.insert(tk.END, "INGREDIENTS\n")
    for row in recipe_contents:
        recipe_contents_text.insert(tk.END, ", ".join(row) + "\n")
    recipe_contents_text.pack()
    
    recipe_instructions_file = f"{recipe_name}_instructions.csv"
    recipe_instructions_path = os.path.join(folder_path, recipe_instructions_file)

    with open(recipe_instructions_path, 'r') as file:
        reader = csv.reader(file)
        recipe_instructions = list(reader)

    recipe_contents_text.insert(tk.END, "\nINSTRUCTIONS\n")
    for row in recipe_instructions:
        recipe_contents_text.insert(tk.END, ", ".join(row) + "\n"+ " "+"\n")
    
    recipe_contents_text = tk.Text(window, width=12, height=8)
    recipe_contents_text.pack()

window = tk.Tk()
window.geometry("500x300")
window.title("How would you like to choose a recipe?")

options = ['Ingredients', 'Cooktime', 'Calories']
selected_option = tk.StringVar(window)
selected_option.set(options[0])

option_menu = tk.OptionMenu(window, selected_option, *options)
option_menu.pack()

# Button to trigger option selection
select_button = tk.Button(window, text="Select", command=on_option_selected)
select_button.pack()

window.mainloop()